In [22]:
from urllib.parse import quote
from urllib import request
import json
import xlwt
import pandas as pd
import numpy as np

In [23]:
# 将返回的poi数据装入集合返回
def hand(poilist, result, adcode, counter):
    pois = result['pois']
    
    for i in range(len(pois)):
        #print(pois[i]['adcode'])
        ad = pois[i]['adcode']
        if ad == adcode:
            #print('in')
            poilist.append(pois[i])
            counter = counter + 1
            
    return counter

In [24]:
# 单页获取pois
def getpoi_page(minlng, maxlat, maxlng, minlat, types, page):
    req_url = poi_search_url + 'key=' + amap_web_key + '&extensions=all&keywords=' + quote(
        types) + '&polygon=' + str(minlng) + ',' + str(maxlat) + '|' + str(maxlng) + ',' + str(minlat) + '&offset=20' + '&page=' + str(
        page) + '&output=json'

    data = ''
    #print(quote(types))
    with request.urlopen(req_url) as f:
        data = f.read()
        data = data.decode('utf-8')
    #print(req_url)
    return data

In [45]:
# 根据grid和分类关键字获取poi数据
def getpois(minlng, maxlat, maxlng, minlat, keywords, adcode, counter):
    #counter = 0
    i = 1
    poilist = []
    
    while True:  # 使用while循环不断分页获取数据
        result = getpoi_page(minlng, maxlat, maxlng, minlat, keywords, i)
        #print(result)
        result = json.loads(result)  # 将字符串转换为json
        #print(result['count'])  
        
        #if result['count'] == 900:
            #generate_grid()
        
        if result['count'] == '0':
            break
        counter = hand(poilist, result, adcode, counter)
            
        i = i + 1
    print('counter='+str(counter))
    return poilist, counter

In [46]:
def combinedf(poilist, df):
    for i in range(len(poilist)):
        types = poilist[i]['type']
        big = str(types).split(";")[0]
        median = str(types).split(";")[1]
        small = str(types).split(";")[2]
        location = poilist[i]['location']
        name = poilist[i]['name']
        lng = str(location).split(",")[0]
        lat = str(location).split(",")[1]
        pname = poilist[i]['pname']
        city = poilist[i]['cityname']
        qu = poilist[i]['adname']
        
        df=df.append(pd.DataFrame({'用地类型大':[big],'用地类型中':[median],'用地类型小':[small],'x':[lng],'y':[lat],'name':[name],'类型':[types],'省':[pname],'市':[city],'区':[qu]}),ignore_index=True)
        print(df)
        return df
    
    #要让df进入recursion

In [47]:
#如果方格数据大于900，进行二次划分
def generate_grids(start_long,start_lat,end_long,end_lat, num, adcode, time):
    
    assert start_long < end_long,'需要从左上到右下设置经度，start的经度应小于end的经度'
    assert start_lat > end_lat,'需要从左上到右下设置纬度，start的纬度应大于end的纬度'
    #assert resolution>0,'resolution应大于0'
    
    centerlng = round(float((end_long+start_long)/2),6)
    centerlat = round(float((end_lat+start_lat)/2),6)
    minlng = round(float(start_long),6)
    maxlng = round(float(end_long),6)
    minlat = round(float(end_lat),6)
    maxlat = round(float(start_lat),6)
    
    book = xlwt.Workbook(encoding='utf-8', style_compression=0)
    sheet = book.add_sheet('GRID', cell_overwrite_ok=True)
    
    sheet.write(0, 0, '起始经度')
    sheet.write(0, 1, '起始纬度')
    sheet.write(0, 2, '终止经度')
    sheet.write(0, 3, '终止纬度')
    
    #first grid -- upper-left
    sheet.write(1, 0, minlng)
    sheet.write(1, 1, maxlat)
    sheet.write(1, 2, centerlng)
    sheet.write(1, 3, centerlat)
    
    #second grid -- upper-right
    sheet.write(2, 0, centerlng)
    sheet.write(2, 1, maxlat)
    sheet.write(2, 2, maxlng)
    sheet.write(2, 3, centerlat)
    
    #third grid -- lower-left
    sheet.write(3, 0, minlng)
    sheet.write(3, 1, centerlat)
    sheet.write(3, 2, centerlng)
    sheet.write(3, 3, minlat)
    
    #fourth grid -- lower-right
    sheet.write(4, 0, centerlng)
    sheet.write(4, 1, centerlat)
    sheet.write(4, 2, maxlng)
    sheet.write(4, 3, minlat)
    
    # 最后，将以上操作保存到指定的Excel文件中
    book.save(r'' + str(time) + "次_GRID_"+ adcode + '_' + str(num) +'.xls')

In [48]:
def read_excel(adcode):
    data = pd.read_excel('GRID_' + adcode + '.xls')
    return data

In [49]:
def more_read_excel(adcode, num, time):
    data = pd.read_excel(str(time) + "次_GRID_"+ adcode + '_' + str(num) +'.xls')
    return data

In [52]:
def recursion(time, data, df):
    for i in range(4):
        record = data.iloc[i]
        minlng = record.iloc[0]
        maxlat = record.iloc[1]
        maxlng = record.iloc[2]
        minlat = record.iloc[3]
        for clas in classes:
            classes_all_pois = []
            counter = 0
            pois_area, counter= getpois(minlng, maxlat, maxlng, minlat, clas, adcode, counter)
            #print('当前城区：' + str(area) + ', 分类：' + str(clas) + ", 总的有" + str(len(pois_area)) + "条数据")
            classes_all_pois.extend(pois_area)
            #print("所有城区的数据汇总，总数为：" + str(len(classes_all_pois)))
            num = i + 1
            if counter < 900:
                
                #write_to_excel(classes_all_pois, clas, num, adcode, time)
                df = combinedf(classes_all_pois, df)
                
                #path = clas + '_' + str(adcode) + '_' + str(num) + '格_' + str(time) + '次' + '.xls'
                #auto_save_file(path)
                print('================分类：'  + str(clas) + str(i+1) + '格' + str(time) + "次写入成功")
            else:
                generate_grids(minlng, maxlat, maxlng, minlat, num, adcode, time)
                print(str(num) + '格 次'+ str(time) + ' 数据大于等于900条，已再次分格')
                
                divide_data = more_read_excel(adcode, num, time)
                time = time + 1
                recursion(time, divide_data, df)
                time = time - 1
                
    return df


In [51]:
poi_search_url = "http://restapi.amap.com/v3/place/polygon?"
classes = ['餐饮服务']
amap_web_key = input('Please input your key: ')
str(amap_web_key)
adcode = input('Please input your adcode: ')
adcode = str(adcode)
data = read_excel(adcode)
time = 1 

df = pd.DataFrame(columns = ['用地类型大','用地类型中','用地类型小','x','y','name','类型','省','市','区'])
recursion(time, data, df)
print('Finish')
df
#8.10 test

Please input your key: c46811a81734366e089404d4b270049e
Please input your adcode: 440106
counter=176
  用地类型大 用地类型中    用地类型小           x          y  name                类型    省  \
0  餐饮服务   中餐厅  广东菜(粤菜)  113.361271  23.207922  鸽皇山庄  餐饮服务;中餐厅;广东菜(粤菜)  广东省   

     市    区  
0  广州市  天河区  
================分类：餐饮服务1格1次写入成功
counter=562
  用地类型大 用地类型中 用地类型小           x          y    name            类型    省    市  \
0  餐饮服务   中餐厅   中餐厅  113.394923  23.200367  恒信水库渔村  餐饮服务;中餐厅;中餐厅  广东省  广州市   

     区  
0  天河区  
================分类：餐饮服务2格1次写入成功
counter=900
3格 次1 数据大于等于900条，已再次分格
counter=876
  用地类型大 用地类型中 用地类型小           x          y         name            类型    省  \
0  餐饮服务   快餐厅   麦当劳  113.315866  23.152667  麦当劳(禺东西路餐厅)  餐饮服务;快餐厅;麦当劳  广东省   

     市    区  
0  广州市  天河区  
================分类：餐饮服务1格2次写入成功
counter=900
2格 次2 数据大于等于900条，已再次分格
counter=408
  用地类型大 用地类型中 用地类型小           x          y         name            类型    省  \
0  餐饮服务   中餐厅   中餐厅  113.349429  23.166246  新一家农庄(天河总店)  餐饮服务;中餐厅;中餐厅  广东省

,用地类型大,用地类型中,用地类型小,x,y,name,类型,省,市,区


In [40]:
df.head()

,用地类型大,用地类型中,用地类型小,x,y,name,类型,省,市,区
